In [1]:
import timeit
start=timeit.default_timer()

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [6]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

## Load files

In [8]:
filename=['brady.txt','butler.txt','cutting.txt','dawson.txt','dyke.txt',
          'hughes.txt','mitchell.txt','richmond.txt','rinehart.txt','stretton.txt']


In [9]:
def load_text(filename):
    file=open(filename,'r')
    text=file.read()
    file.close
    return text

In [10]:
def clean_text(text):
    text=text.lower()
    text=text.replace("ï»¿","")
    text=text.replace(","," ")
    text=text.replace("\n"," ")
    text=text.replace("*"," ")
    text=text.replace(":"," ")
    text=text.replace(":"," ")
    text=re.sub("[\(\[].*?[\)\]]"," ",text)
    text = re.sub(r'--',' ',text)
    text= text.replace("http //www.gutenberg.net"," ")
    text=text.replace("www.gutenberg.net"," ")
    text=text.replace("14606-h.htm"," ")

    text=text.replace("i."," ")
    text=text.replace("ii."," ")
    text=text.replace("iii."," ")
    text=text.replace("iv."," ")
    text=text.replace("v."," ")
    text=text.replace("vi."," ")
    text=text.replace("vi."," ")
    text=text.replace("vii."," ")
    text=text.replace("viii."," ")
    text=text.replace("ix."," ")
    text=text.replace("x."," ")
    text=text.replace("xi."," ")

    text=text.replace('\'',"")
    text=text.replace('/',"")
    text=text.replace('14606-h.z',"")
    text=text.replace('iso-646-us',"")
    text=text.replace('?'," ")
    text = re.sub('\S*@\S*\s?',' ',text)
    text=text.replace('"'," ")

    return text

In [11]:
text_list=[]
len_list=[None]*(len(filename)+1)
len_list[0]=0
for i in range(0,len(filename)):
    text=load_text(filename[i])
    text=clean_text(text)
    temp_list=text.split(".")
    len_list[i+1]=len_list[i]+len(temp_list)
    text_list=text_list+temp_list

In [12]:
len(text_list)

6691

In [13]:
len_list

[0, 936, 1574, 2001, 2931, 3504, 4050, 4682, 5286, 6074, 6691]

In [14]:
df=pd.DataFrame(index=range(len(text_list)),columns=['author'])
for i in range(0,(len(len_list)-1)):
    author=filename[i].replace('.txt','')
    x=len_list[i]
    y=len_list[i+1]
    df['author'][x:y]=author

In [15]:
len(text_list)

6691

In [16]:
df.shape

(6691, 1)

In [17]:
len_list

[0, 936, 1574, 2001, 2931, 3504, 4050, 4682, 5286, 6074, 6691]

In [18]:
len(text_list)

6691

In [19]:
vectorizer=CountVectorizer(stop_words="english", 
                           analyzer='word', 
                           ngram_range=(1, 3), 
                           max_df=1.0, 
                           min_df=1, 
                           max_features=500)

In [20]:
train_data_features = vectorizer.fit_transform(text_list)
train_data_features = train_data_features.toarray()
train_data_features.shape

(6691, 500)

In [21]:
X=train_data_features
y=df['author']

In [22]:
rfc = ensemble.RandomForestClassifier()
scores=cross_val_score(rfc,X,y,cv=4,scoring='accuracy')
scores.mean()

0.2983490219447145

In [23]:
lr = LogisticRegression()
scores=cross_val_score(lr,X,y,cv=4,scoring='accuracy')
scores.mean()

0.333301471985916